##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> </a><a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback">Посмотреть на TensorFlow.org</a><a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback">View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> </a><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb">Запустить в Google Colab</a><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb">Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> </a><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb">Посмотреть источник на GitHub</a><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb">View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> </a><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb">Скачать блокнот</a><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb">Download notebook</a>
  </td>
</table>

# Пользовательские обратные вызовы Keras

Пользовательский обратный вызов - это мощный инструмент для настройки поведения модели Keras во время обучения, оценки или вывода, включая чтение / изменение модели Keras. Примеры включают в себя `tf.keras.callbacks.TensorBoard` где результаты и результаты обучения могут быть экспортированы и визуализированы с помощью TensorBoard, или `tf.keras.callbacks.ModelCheckpoint` где модель автоматически сохраняется во время обучения, и многое другое. В этом руководстве вы узнаете, что такое обратный вызов Keras, когда он будет вызываться, что он может делать и как вы можете создать свой собственный. В конце этого руководства будут продемонстрированы демонстрации создания нескольких простых приложений обратного вызова, которые помогут вам начать свой собственный обратный вызов.

## Настроить

In [0]:
import tensorflow as tf

## Введение в обратные вызовы Keras

В Keras `Callback` - это класс Python, предназначенный для использования в подклассах для обеспечения определенной функциональности, с набором методов, вызываемых на различных этапах обучения (включая начало и конец серии / эпохи), тестирования и прогнозирования. Обратные вызовы полезны, чтобы получить представление о внутренних состояниях и статистике модели во время обучения. Вы можете передать список обратных вызовов (в качестве `callbacks` аргумента ключевого слова) любому из `tf.keras.Model.fit()` , `tf.keras.Model.evaluate()` и `tf.keras.Model.predict()` . Методы обратных вызовов будут вызываться на разных этапах обучения / оценки / вывода.

Чтобы начать, давайте импортируем тензор потока и определим простую модель Sequential Keras:

In [0]:
# Define the Keras model to add callbacks to
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

Затем загрузите данные MNIST для обучения и тестирования из API наборов данных Keras:

In [0]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

Теперь определите простой пользовательский обратный вызов для отслеживания начала и конца каждого пакета данных. Во время этих вызовов он печатает индекс текущего пакета.

In [0]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_begin(self, batch, logs=None):
    print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_end(self, batch, logs=None):
    print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

Предоставление обратного вызова для методов модели, таких как `tf.keras.Model.fit()` обеспечивает вызов методов на следующих этапах:

In [0]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

## Модельные методы, которые принимают обратные вызовы

Пользователи могут предоставить список обратных вызовов для следующих методов `tf.keras.Model` :

#### [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) , [`fit_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator)

Обучает модель для фиксированного числа эпох (итерации по набору данных или данные, полученные от пакета к генератору Python).

#### [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) , [`evaluate_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate_generator)

Оценивает модель для данных или генератора данных. Выводит значения потерь и метрики из оценки.

#### [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict) , [`predict_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict_generator)

Генерирует выходные прогнозы для входных данных или генератора данных.

In [0]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

## Обзор методов обратного вызова

### Общие методы обучения / тестирования / прогнозирования

Для обучения, тестирования и прогнозирования предусмотрены следующие методы для переопределения.

#### `on_(train|test|predict)_begin(self, logs=None)`

Вызывается в начале `fit` / `evaluate` / `predict` .

#### `on_(train|test|predict)_end(self, logs=None)`

Вызывается в конце `fit` / `evaluate` / `predict` .

#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`

Вызывается прямо перед обработкой партии во время обучения / тестирования / прогнозирования. В этом методе `logs` - это диктовка с ключами `batch` и `size` , представляющими номер текущей партии и размер партии.

#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`

Вызывается в конце обучения / тестирования / прогнозирования партии. В этом методе `logs` - это диктат, содержащий результат метрик с отслеживанием состояния.

### Обучение конкретным методам

Кроме того, для обучения предоставляются следующие.

#### on_epoch_begin (self, epoch, logs = None)

Вызывается в начале эпохи во время тренировки.

#### on_epoch_end (self, epoch, logs = None)

Вызывается в конце эпохи во время тренировки.

### Использование `logs` dict

`logs` dict содержит значение потерь и все метрики в конце партии или эпохи. Пример включает в себя потери и среднюю абсолютную ошибку.

In [0]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

Точно так же можно обеспечить обратные вызовы в `evaluate()` .

In [0]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

## Примеры приложений обратного вызова Keras

Следующий раздел поможет вам в создании простых приложений обратного вызова.

### Ранняя остановка с минимальными потерями

Первый пример демонстрирует создание `Callback` который останавливает обучение Keras, когда достигается минимум потерь путем изменения атрибута `model.stop_training` (логическое значение). При желании пользователь может предоставить аргумент `patience` чтобы указать, сколько эпох обучение должно ждать, прежде чем оно в конечном итоге останавливается.

`tf.keras.callbacks.EarlyStopping` предоставляет более полную и общую реализацию.

In [0]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [0]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

### Планирование скорости обучения

One thing that is commonly done in model training is changing the learning rate as more epochs have passed. Keras backend exposes `get_value` API which can be used to set the variables. In this example, we're showing how a custom Callback can be used to dynamically change the learning rate.

Примечание: это всего лишь пример реализации. Посмотрите `callbacks.LearningRateScheduler` и `keras.optimizers.schedules` для более общих реализаций.

In [0]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [0]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])

### Стандартные обратные вызовы Keras

Обязательно ознакомьтесь с существующими обратными вызовами Keras, [посетив документацию по API](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks) . Приложения включают регистрацию в CSV, сохранение модели, визуализацию на TensorBoard и многое другое.